In [ ]:
import requests as rq
import pandas as pd
from scipy.stats import poisson
year_prev = '2021-22'
year = '2022-23'

# tracking class
class Stat:
  # exp decay rate
  def __init__(self, exp, stat, total):
    self.exp = exp
    self.stat = stat
    self.total = total
  def rate(self):
    if(self.total == 0):
      return 0.0
    return self.stat / self.total
  def update(self, stat, total):
    self.stat *= self.exp
    self.total *= self.exp
    self.stat += stat
    self.total += total

# converts in_report to comma-separated id values
def to_id(report, team_name, player_name):
  if(report == '0'):
    return '0'
  names = report.split(",")
  new_names = []
  for name in names:
    new_names.append(get_player_id(year, team_name, name))
  return ",".join(new_names)

# returns team id from team abbrev
# 'ATL','BKN','BOS','CHA','CHI','CLE','DAL','DEN','DET','GSW','HOU','IND','LAC','LAL','MEM','MIA','MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHX','POR','SAC','SAS','TOR','UTA','WAS'
teams_response = rq.get('https://api.pbpstats.com/get-teams/nba')
teams = teams_response.json()
def get_team_id(team_name):
  for team in teams['teams']:
    if(team['text'] == team_name):
      return team['id']

# all nba player ids
players_url = 'https://api.pbpstats.com/get-team-players-for-season'
def get_player_id(season, team_name, player_name):
  players_params = {
    'Season': season,
    'SeasonType': 'Regular Season',
    'TeamId': get_team_id(team_name)
  }
  players_response = rq.get(players_url, params=players_params)
  players = players_response.json()
  for player in players['players'].items():
    if(player[1] == player_name):
      return player[0]

In [ ]:
# league averages
url = 'https://api.pbpstats.com/get-team-leverage-summary/nba'
params = {
    'Season': year,
    'Leverage': 'Medium'
}
r = rq.get(url, params=params)
r_json = r.json()
avg_pace = 0
total_possessions = 0
for d in r_json['results']:
  total_possessions += d['off_poss']
  avg_pace += d['seconds_per_poss_off'] * d['off_poss']
avg_pace /= total_possessions
# average possessions per game
url = 'https://api.pbpstats.com/get-totals/nba'
params = {
    'Season': year,
    'SeasonType': 'Regular Season',
    'Type': 'Team'
}
r = rq.get(url, params=params)
r_json = r.json()
league_stats = r_json["single_row_table_data"]
league_opace = league_stats['SecondsPerPossOff']
poss_per_game = league_stats['Pace']
efg_pct = league_stats['EfgPct']
ft_per_100 = league_stats['FTA'] / league_stats['OffPoss'] * 100
oreb_per_100 = league_stats['OffRebounds'] / league_stats['OffPoss'] * 100
dreb_per_100 = league_stats['DefRebounds'] / league_stats['DefPoss'] * 100
to_per_100 = league_stats['LiveBallTurnovers'] / league_stats['OffPoss'] * 100
steals_per_100 = league_stats['Steals'] / league_stats['DefPoss'] * 100

In [ ]:
# Example exponential decay constants
# Used to capture "momentum" in streaks, since some fields are inherently more volatile
shooting_exp = 0.985
exp = 0.9925
minutes_exp = 0.97
url1 = 'https://api.pbpstats.com/get-game-logs/nba'
url2 = 'https://api.pbpstats.com/get-wowy-stats/nba'
# returns 2pm,3pm,ftm,reb,assists,turnovers,blocks,steals
def projections(team_name, opp_team_name):
  team_id = get_team_id(team_name)
  opp_team_id = get_team_id(opp_team_name)
  players = in_report[team_name].split(',')# opponent game dates
  opp_in_id = to_id(in_report[opp_team_name], opp_team_name, 'x')
  opp_out_id = to_id(out_report[opp_team_name], opp_team_name, 'x')
  params = {
    'Season': year,
    'SeasonType': 'Regular Season',
    'EntityId': opp_team_id,
    'EntityType': 'Team'
  }
  r1 = rq.get(url1, params=params)
  r1_json = r1.json()
  games1 = r1_json['multi_row_table_data']
  wowy_params = {
    '0Exactly0DidNotPlayInGame': opp_in_id, # number(Exactly|GreaterThan|LessThan)number(OnFloor|OffFloor|PlayedInGame|DidNotPlayInGame|Started|CameOffBench)
    '0Exactly0OnFloor': opp_out_id,
    'TeamId': opp_team_id,
    'Season': year,
    'SeasonType': 'Regular Season',
    'Type': 'Opponent'
  }
  # opponent stats
  opp_pace = Stat(exp, 0, 0)
  opp_efg_pct = Stat(exp, 0, 0)
  opp_fta_100 = Stat(exp, 0, 0)
  opp_dreb_100 = Stat(exp, 0, 0)
  opp_oreb_100 = Stat(exp, 0, 0)
  opp_to_100 = Stat(exp, 0, 0)
  opp_steal_100 = Stat(exp, 0, 0)
  for game in games1:
    wowy_params['FromDate'] = game['Date']
    wowy_params['ToDate'] = game['Date']
    wowy_response = rq.get(url2, params=wowy_params)
    wowy = wowy_response.json()
    team_stats = wowy['single_row_table_data']
    if(len(team_stats) > 0):
      opp_pace.update((team_stats['SecondsPerPossOff'] if 'SecondsPerPossOff' in team_stats else 0) * (team_stats['OffPoss'] if 'OffPoss' in team_stats else 0), team_stats['OffPoss'] if 'OffPoss' in team_stats else 0)
      wowy_params['Type'] = 'Opponent'
      wowy_response = rq.get(url2, params=wowy_params)
      wowy = wowy_response.json()
      opp_stats = wowy['single_row_table_data']
      opp_efg_pct.update((opp_stats['EfgPct'] if 'EfgPct' in opp_stats else 0) * ((opp_stats['FG2A'] if 'FG2A' in opp_stats else 0) + (opp_stats['FG3A'] if 'FG3A' in opp_stats else 0)), (opp_stats['FG2A'] if 'FG2A' in opp_stats else 0) + (opp_stats['FG3A'] if 'FG3A' in opp_stats else 0))
      opp_fta_100.update(opp_stats['FTA'] if 'FTA' in opp_stats else 0, opp_stats['OffPoss'])
      opp_dreb_100.update(opp_stats['DefRebounds'] if 'DefRebounds' in opp_stats else 0, opp_stats['DefPoss'] if 'DefPoss' in opp_stats else 0)
      opp_oreb_100.update(opp_stats['OffRebounds'] if 'OffRebounds' in opp_stats else 0, opp_stats['OffPoss'] if 'OffPoss' in opp_stats else 0)
      wowy_params['Type'] = 'Team'
      wowy_response = rq.get(url2, params=wowy_params)
      wowy = wowy_response.json()
      team_stats = wowy['single_row_table_data']
      opp_to_100.update(team_stats['LiveBallTurnovers'] if 'LiveBallTurnovers' in team_stats else 0, team_stats['OffPoss'] if 'OffPoss' in team_stats else 0)
      opp_steal_100.update(team_stats['Steals'] if 'Steals' in team_stats else 0, team_stats['DefPoss'] if 'DefPoss' in team_stats else 0)
  for player in players:
    # in/out reports converted to id
    in_id = to_id(in_report[team_name], team_name, player)
    out_id = to_id(out_report[team_name], team_name, 'x')

    # player stats
    player_poss_100 = Stat(minutes_exp, 0, 0)
    player_wowy_poss_100 = Stat(minutes_exp, 0, 0)
    player_2pm_100 = Stat(shooting_exp, 0, 0)
    player_3pm_100 = Stat(shooting_exp, 0, 0)
    player_ftm_100 = Stat(shooting_exp, 0, 0)
    player_dreb_100 = Stat(exp, 0, 0)
    player_oreb_100 = Stat(exp, 0, 0)
    player_assist_100 = Stat(exp, 0, 0)
    player_to_100 = Stat(exp, 0, 0)
    player_block_100 = Stat(exp, 0, 0)
    player_steal_100 = Stat(exp, 0, 0)

    # team stats
    team_pace = Stat(exp, 0, 0)

    # team game dates
    params = {
      'Season': year,
      'SeasonType': 'Regular Season',
      'EntityId': team_id,
      'EntityType': 'Team'
    }
    r1 = rq.get(url1, params=params)
    r1_json = r1.json()
    games1 = r1_json['multi_row_table_data']
    wowy_params = {
      '0Exactly0DidNotPlayInGame': in_id, # number(Exactly|GreaterThan|LessThan)number(OnFloor|OffFloor|PlayedInGame|DidNotPlayInGame|Started|CameOffBench)
      '0Exactly0OnFloor': out_id,
      'TeamId': team_id,
      'Season': year,
      'SeasonType': 'Regular Season',
      'Type': 'Team',
      'FromDate': '2023-03-16',
      'ToDate': '2023-03-16'
    }
    for game in games1:
      # player stats
      wowy_params['Type'] = 'Team'
      wowy_params['FromDate'] = game['Date']
      wowy_params['ToDate'] = game['Date']
      wowy_response = rq.get(url2, params=wowy_params)
      wowy = wowy_response.json()
      team_stats = wowy['single_row_table_data']
      if(len(team_stats) > 0):
        team_pace.update((team_stats['SecondsPerPossOff'] if 'SecondsPerPossOff' in team_stats else 0) * (team_stats['OffPoss'] if 'OffPoss' in team_stats else 0), team_stats['OffPoss'] if 'OffPoss' in team_stats else 0)
        wowy_params['Type'] = 'Player'
        temp = wowy_params.pop('0Exactly0OnFloor')
        wowy_response = rq.get(url2, params=wowy_params)
        wowy = wowy_response.json()
        player_stats = wowy['multi_row_table_data']
        for player_stat in player_stats:
          if(player_stat['Name'] == player):
            player_poss_100.update(player_stat['OffPoss'] if 'OffPoss' in player_stat else 0, team_stats['OffPoss'] if 'OffPoss' in team_stats else 0)
        wowy_params['0Exactly0OnFloor'] = temp
        wowy_response = rq.get(url2, params=wowy_params)
        wowy = wowy_response.json()
        player_stats = wowy['multi_row_table_data']
        for player_stat in player_stats:
          if(player_stat['Name'] == player):
            if('OffPoss' in player_stat):
              player_wowy_poss_100.update(player_stat['OffPoss'], team_stats['OffPoss'])
              player_2pm_100.update(player_stat['FG2M'] if 'FG2M' in player_stat else 0, player_stat['OffPoss'])
              player_3pm_100.update(player_stat['FG3M'] if 'FG3M' in player_stat else 0, player_stat['OffPoss'])
              player_ftm_100.update(player_stat['FtPoints'] if 'FtPoints' in player_stat else 0, player_stat['OffPoss'])
              player_oreb_100.update(player_stat['OffRebounds'] if 'OffRebounds' in player_stat else 0, player_stat['OffPoss'])
              player_assist_100.update(player_stat['Assists'] if 'Assists' in player_stat else 0, player_stat['OffPoss'])
              player_to_100.update(player_stat['Turnovers'] if 'Turnovers' in player_stat else 0, player_stat['OffPoss'])
            if('DefPoss' in player_stat):
              player_dreb_100.update(player_stat['DefRebounds'] if 'DefRebounds' in player_stat else 0, player_stat['DefPoss'])
              player_block_100.update(player_stat['Blocks'] if 'Blocks' in player_stat else 0, player_stat['DefPoss'])
              player_steal_100.update(player_stat['Steals'] if 'Steals' in player_stat else 0, player_stat['DefPoss'])

    # returns 2pm,3pm,ftm,reb,assists,turnovers,blocks,steals
    proj_poss = (min(max(player_poss_100.rate(), player_wowy_poss_100.rate()), .825) * poss_per_game * (team_pace.rate() + opp_pace.rate())) / (2 * league_opace)
    player_2pm = player_2pm_100.rate() * proj_poss * opp_efg_pct.rate() / efg_pct
    player_3pm = player_3pm_100.rate() * proj_poss * opp_efg_pct.rate() / efg_pct
    player_ftm = player_ftm_100.rate() * proj_poss * 100 * opp_fta_100.rate() / ft_per_100
    player_reb = player_dreb_100.rate() * proj_poss * 100 * opp_dreb_100.rate() / dreb_per_100 + player_oreb_100.rate() * proj_poss * opp_oreb_100.rate() / oreb_per_100
    player_ast = player_assist_100.rate() * proj_poss * opp_efg_pct.rate() / efg_pct
    player_to = player_to_100.rate() * proj_poss * 100 * opp_steal_100.rate() / steals_per_100
    player_block = player_block_100.rate() * proj_poss
    player_steal = player_steal_100.rate() * proj_poss * 100 * opp_to_100.rate() / to_per_100
    print(player)
    print(','.join([str(round(player_2pm, 4)), str(round(player_3pm, 4)), str(round(player_ftm, 4)), str(round(player_reb, 4)),
                    str(round(player_ast, 4)), str(round(player_to, 4)), str(round(player_block, 4)), str(round(player_steal, 4))]))
  return ''

In [ ]:
# all players ruled out that are relevant
out_report = {}
# all players in that are relevant
in_report = {}

def project(boolean, team_1, team_2, in_report_1, out_report_1, in_report_2, out_report_2):
  if(not boolean): return ''
  in_report[team_1] = in_report_1
  in_report[team_2] = in_report_2
  out_report[team_1] = out_report_1
  out_report[team_2] = out_report_2
  projections(team_1, team_2)
  projections(team_2, team_1)
  return ''
# Player ID / name tester
# print(get_player_id(year, 'NYK', 'RJ Barrett'))
project(False, 'ORL', 'WAS',
        'Wendell Carter Jr.,Paolo Banchero,Franz Wagner,Gary Harris,Markelle Fultz',
        'Jalen Suggs',
        'Bradley Beal,Kristaps Porzingis,Monte Morris,Daniel Gafford',
        '0')

project(True, 'NOP', 'SAS',
        'Herbert Jones,Brandon Ingram,CJ McCollum,Trey Murphy III',
        '0',
        'Keldon Johnson',
        'Jeremy Sochan,Devin Vassell')

project(True, 'DET', 'ATL',
        'James Wiseman,Jaden Ivey,Killian Hayes',
        'Bojan Bogdanovic,Jalen Duren,Alec Burks,Cade Cunningham',
        'De\'Andre Hunter,Clint Capela',
        '0')

project(True, 'BKN', 'CLE',
        'Cameron Johnson,Nic Claxton,Mikal Bridges,Spencer Dinwiddie',
        '0',
        'Darius Garland,Evan Mobley,Donovan Mitchell',
        'Jarrett Allen')

Herbert Jones
3.2245,0.7931,1.9508,2.6622,2.5199,1.1057,0.5259,1.7509
Brandon Ingram
7.4667,1.5763,4.6433,4.5486,5.3653,2.8332,0.4505,0.7157
CJ McCollum
5.1759,2.632,2.3594,3.6827,6.2847,1.9126,0.5238,1.0133
Trey Murphy III
2.0068,2.2909,1.4164,2.8877,1.4677,0.5183,0.5398,1.0768
Keldon Johnson
8.1812,1.7681,5.4654,5.2841,3.4993,3.3355,0.156,0.6283
Jaden Ivey
3.5321,2.6684,4.9394,3.7555,5.7344,3.9082,0.2038,0.9334
Killian Hayes
3.8985,1.2972,0.7214,1.9761,9.5431,3.0954,0.5388,1.7139
De'Andre Hunter
4.3266,1.4317,3.0329,3.422,1.5376,1.0254,0.2625,0.489
Clint Capela
5.3701,0.0,1.1683,6.7641,0.9099,0.6067,1.1456,0.6746
James Wiseman
8.637,0.3824,0.7238,11.4969,0.744,2.0621,1.0593,0.3556
Jaden Ivey
2.4557,1.6558,3.5945,2.3907,6.1713,4.5591,0.0,0.0
Killian Hayes
4.0951,0.5254,0.9792,2.8637,13.3313,3.8165,1.4537,1.4824
De'Andre Hunter
4.3813,1.4498,2.7422,3.4045,1.5571,0.8666,0.27,0.5296
Clint Capela
5.4381,0.0,1.0563,6.7306,0.9214,0.5127,1.1784,0.7306
Cameron Johnson
3.0506,2.381,3.5591,4.93

''